# Airflow DAG Generation: Evaluation

This notebook evaluates the quality of Airflow DAGs generated by different models, comparing a baseline model (Qwen 2.5 1.5B Instruct) against a fine-tuned version.

## 📋 Setup Note

**This notebook is designed to run locally** (not in Colab). To set it up:

```bash
# From project root
pip install -e ".[research]"
pip install jupyter

# Launch Jupyter and select the venv kernel
jupyter notebook
```

Make sure you're using the Python kernel from your virtual environment to ensure all imports work correctly.

---

## 📊 Performance Summary (Fine-tuned vs Base)

Our evaluation reveals significant improvements across multiple dimensions:

### Key Improvements
- **Syntax Validity**: ~8% reduction in syntactically invalid DAGs
- **Modern Patterns**: Strong adoption of latest Airflow syntax and operators (TaskFlow API, modern decorators)
- **Reduced Hallucinations**: Significantly fewer instances of invented imports or non-existent operators
- **Error Distribution Alignment**: The fine-tuned model's error patterns closely match real-world DAG distributions

### Notable Observations
- **Base Model**: Often generates deprecated patterns (e.g., legacy operators from Airflow 1.x)
- **Fine-tuned Model**: Occasionally hallucinates internal testing libraries seen in training data, but far less than general hallucinations in the base model
- **Syntax Accuracy**: Fine-tuned model shows consistent adherence to Python and Airflow syntax rules

---

## 🔬 Evaluation Methodology

We employ a two-pronged evaluation approach to assess both structural correctness and semantic quality:

### 1. Parser-Based Evaluation (Structural Analysis)

Uses a custom AST-based validator (`DAGValidator`) to check:
- **Syntax Correctness**: Valid Python syntax, no parse errors
- **Task ID Validation**: Unique task IDs, proper naming conventions (alphanumeric, dashes, dots, underscores)
- **Dependency Analysis**: Detection of circular dependencies in task graphs
- **DAG Structure**: Proper DAG instantiation, task definitions, and relationships

**Advantages**: Fast, deterministic, catches critical structural errors that would prevent DAG execution.

### 2. LLM-Based Evaluation (Semantic Analysis)

Uses Claude 4.5 Sonnet via the Batch API to evaluate:
- **Correctness** (1-5): Does the generated DAG logically implement the user's request?
- **Completeness** (1-5): Are all necessary imports, arguments, and task dependencies present?
- **Best Practices** (1-5): Does it follow Airflow conventions and modern patterns?

**Advantages**: Captures semantic quality, intent alignment, and code quality aspects that structural analysis misses.

**Note**: LLM evaluation requires an Anthropic API key and incurs costs. Results are saved for reproducibility.

## 1. Setup and Configuration

In [65]:
import json
import glob
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from typing import List, Dict, Any
import re

# hf
from datasets import load_dataset

# Import from installed packages
from research.lib.batch_processor import ClaudeBatchProcessor
from airflow_net.validation import DAGValidator

# Set visualization style
sns.set_theme(style="whitegrid", context="notebook", palette="viridis")

In [48]:
# API Key Configuration
import os

ANTHROPIC_API_KEY = os.environ.get('ANTHROPIC_API_KEY')

if not ANTHROPIC_API_KEY:
    print("WARNING: No ANTHROPIC_API_KEY found in environment. LLM evaluation steps will be skipped.")
    print("To enable LLM evaluation, set the ANTHROPIC_API_KEY environment variable.")

## 2. Load Data
We load the generated DAGs from the JSONL artifacts produced by the inference step and the test dataset containing the ground truth DAG files.

In [49]:
# Define paths to artifacts (relative to notebook location)
ARTIFACTS_DIR = Path("../../artifacts/finetuning/01_inference_results").resolve()

print(f"Looking for artifacts in: {ARTIFACTS_DIR}")

# Find latest inference files
base_files = list(ARTIFACTS_DIR.glob("base_model_outputs*.jsonl"))
finetuned_files = list(ARTIFACTS_DIR.glob("finetuned_model_outputs*.jsonl"))

if not base_files or not finetuned_files:
    print("WARNING: Could not find one or both inference result files.")
    print(f"Available files: {list(ARTIFACTS_DIR.glob('*.jsonl'))}")
    BASE_MODEL_FILE = None
    FINETUNED_MODEL_FILE = None
else:
    # Take the most recent one
    BASE_MODEL_FILE = sorted(base_files)[-1]
    FINETUNED_MODEL_FILE = sorted(finetuned_files)[-1]
    print(f"Selected Baseline: {BASE_MODEL_FILE.name}")
    print(f"Selected Fine-tuned: {FINETUNED_MODEL_FILE.name}")

Looking for artifacts in: /Users/andreatamburri/Desktop/airflowNet/research/artifacts/finetuning/01_inference_results
Selected Baseline: base_model_outputs_20251217_151724.jsonl
Selected Fine-tuned: finetuned_model_outputs_20251217_151724.jsonl


In [52]:
def load_jsonl(file_path):
    """Load JSONL file and extract code from messages format"""
    if not file_path or not file_path.exists():
        return []
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                entry = json.loads(line)
                # Extract assistant content from messages
                messages = entry.get('messages', [])
                user_content = ''
                assistant_content = ''
                
                for msg in messages:
                    if msg['role'] == 'user':
                        user_content = msg['content']
                    elif msg['role'] == 'assistant':
                        assistant_content = msg['content']
                    if msg['role'] == 'system':
                        system_content = msg['content']
                
                data.append({
                    'system': system_content,
                    'prompt': user_content,
                    'code': assistant_content,
                    'metadata': entry.get('metadata', {})
                })
    return data

baseline_data = load_jsonl(BASE_MODEL_FILE)
finetuned_data = load_jsonl(FINETUNED_MODEL_FILE)

print(f"Loaded {len(baseline_data)} baseline samples.")
print(f"Loaded {len(finetuned_data)} fine-tuned samples.")

Loaded 412 baseline samples.
Loaded 412 fine-tuned samples.


In [56]:
def parse_hf_ds(dataset):
    """Convert HuggingFace dataset to the same structure as load_jsonl function"""
    data = []

    for entry in dataset:
        messages = entry.get('messages', [])
        system_content = ''
        user_content = ''
        assistant_content = ''

        for msg in messages:
            if msg['role'] == 'system':
                system_content = msg['content']
            elif msg['role'] == 'user':
                user_content = msg['content']
            elif msg['role'] == 'assistant':
                assistant_content = msg['content']

        data.append({
            'system': system_content,
            'prompt': user_content,
            'code': assistant_content,
            'metadata': entry.get('metadata', {})
        })

    return data

dataset = load_dataset(
    "andrea-t94/airflow-dag-dataset",
    split="test",
    download_mode="reuse_cache_if_exists"  # Use cached version if available
)

ground_data = parse_hf_ds(dataset)
print(f"Loaded {len(ground_data)} ground truth samples.")

Generating train split:   0%|          | 0/7414 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/412 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/412 [00:00<?, ? examples/s]

Loaded 412 ground truth samples.


In [57]:
def evaluate_parser_results(data, model_name):
    """Evaluate DAG code using the DAGValidator"""
    results = []
    validator = DAGValidator()
    
    for entry in data:
        code = entry.get('code', '')
        
        # Use DAGValidator.validate_content which returns List[ValidationError]
        errors = validator.validate_content(code, source_name="<generated>")
        
        # Extract error information
        is_valid = len(errors) == 0
        error_messages = [str(e) for e in errors]
        error_types = [e.error_type for e in errors]
        
        results.append({
            'model': model_name,
            'is_valid': is_valid,
            'error_count': len(errors),
            'errors': '; '.join(error_messages) if error_messages else '',
            'has_syntax_error': any('SYNTAX_ERROR' in et or 'PARSE_ERROR' in et for et in error_types),
            'has_duplicate_task': any('DUPLICATE_TASK_ID' in et for et in error_types),
            'has_invalid_task_id': any('INVALID_TASK_ID' in et for et in error_types),
            'has_circular_dependency': any('CIRCULAR_DEPENDENCY' in et for et in error_types)
        })
    return pd.DataFrame(results)

df_ground = evaluate_parser_results(ground_data, 'Ground')
df_base = evaluate_parser_results(baseline_data, 'Baseline')
df_fine = evaluate_parser_results(finetuned_data, 'Fine-tuned')
df_parser = pd.concat([df_ground, df_base, df_fine], ignore_index=True)

# Display summary statistics
summary = df_parser.groupby('model').agg(
    valid_rate=('is_valid', 'mean'),
    syntax_errors=('has_syntax_error', 'mean'),
    duplicate_tasks=('has_duplicate_task', 'mean'),
    invalid_task_ids=('has_invalid_task_id', 'mean'),
    circular_deps=('has_circular_dependency', 'mean')
) * 100

print("Parser Evaluation Results (%):")
display(summary.round(2))


Parser Evaluation Results (%):


,valid_rate,syntax_errors,duplicate_tasks,invalid_task_ids,circular_deps
model,,,,,
Baseline,95.87,3.16,0.00,0.97,0.00
Fine-tuned,71.12,27.91,0.49,0.00,0.73
Ground,79.61,16.99,0.73,0.00,0.24


## 4. LLM-Based Evaluation with Claude
Using Claude 4.5 Sonnet to score DAGs on Correctness, Completeness, and Best Practices.

In [ ]:
EVAL_PROMPT_TEMPLATE = """
You are an expert Senior Apache Airflow Architect. Evaluate the following Airflow DAG code generated by an AI model based on a user instruction.

### Scoring Criteria & Examples

**1. Idiomatic Airflow (Score 0 or 1)**
* **Score 1 (Pass):** Uses specific Providers and Operators designed for the task.
    * *Example:* `from airflow.providers.snowflake.operators.snowflake import SnowflakeOperator`
* **Score 0 (Fail):** Relies on generic "Pythonic" patterns where it wraps logic in a `PythonOperator` + Hook instead of using the native Operator.
    * *Example:* `def load(): hook = SnowflakeHook(...) \n PythonOperator(python_callable=load ...)`

**2. No Hallucination/Leakage (Score 0 or 1)**
* **Score 1 (Pass):** Code is clean, production-ready, and uses only standard Airflow libraries.
* **Score 0 (Fail):** Code imports internal testing modules or includes test harness boilerplate.
    * *Example:* `from tests_common.test_utils.system_tests import get_test_run`
    * *Example:* `test_run = get_test_run(dag)`

**3. Instruction Adherence (Score 0 or 1)**
* **Score 1 (Pass):** Fulfills the specific business logic requested (e.g., "load data AND validate").
* **Score 0 (Fail):** Misses a key step of the instruction.

---

### Task
USER INSTRUCTION:
{instruction}

DAG CODE:
```python
{dag_content}
```


Evaluate the code based on the criteria above. Return valid JSON only.

{{{{
  "idiomatic_airflow": {{{{ "score": 0, "reasoning": "..." }}}}
  "no_hallucination": {{{{ "score": 0, "reasoning": "..." }}}},
  "instruction_adherence": {{{{ "score": 0, "reasoning": "..." }}}}
}}}}
"""

def prepare_llm_batch_requests(dags: List[Dict],
                                model_name: str, 
                                prompt_template: str = EVAL_PROMPT_TEMPLATE) -> List[Dict]:
    """Prepare batch requests for Claude LLM evaluation.
    
    Includes all DAGs with DAG generation requests (even if they failed parser validation).
    """
    batch_requests = []
    
    print(f"Preparing requests for all {len(dags)} DAGs...")
    
    skipped_non_dag = 0
    
    for idx, dag_record in enumerate(dags):
        
        system_message = dag_record.get('system', '')
        # Only include DAG generation requests (filter out other types)
        if not system_message.startswith("You are an expert Apache Airflow developer"):
            skipped_non_dag += 1
            continue
        
        # Create prompt
        prompt = prompt_template.format(
            dag_content=dag_record.get('code', ''), 
            instruction=dag_record.get('prompt', '')
        )
        
        # Create batch request
        request = {
            "custom_id": f"{model_name}_{idx}",
            "params": {
                "model": "claude-sonnet-4-5-20250929",  # Claude Sonnet 4.5
                "max_tokens": 2000,
                "temperature": 0.0,
                "messages": [{"role": "user", "content": prompt}]
            }
        }
        
        batch_requests.append(request)
    if skipped_non_dag > 0:
        print(f"Skipped {skipped_non_dag} non-DAG generation requests")
    
    return batch_requests

print("Preparing LLM evaluation batch requests (including DAGs that failed parser validation)...")


if ANTHROPIC_API_KEY:
    processor = ClaudeBatchProcessor(api_key=ANTHROPIC_API_KEY)
    
    # Prepare batches for full datasets
    print(f"Preparing evaluation batches...")
    ground_batch_requests = prepare_llm_batch_requests(
    baseline_data, 
    "ground"
    )
    baseline_batch_requests = prepare_llm_batch_requests(
    baseline_data, 
    "baseline"
    )
    finetuned_batch_requests = prepare_llm_batch_requests(
        finetuned_data,
        "finetuned"
    )
    all_batch_requests = ground_batch_requests + baseline_batch_requests + finetuned_batch_requests
    
    print(f"✓ Prepared {len(all_batch_requests)} evaluation requests ({len(ground_batch_requests)} ground + {len(baseline_batch_requests)} baseline + {len(finetuned_batch_requests)} fine-tuned)")
else:
    print("⚠️ Skipping LLM evaluation setup:")
    if not ANTHROPIC_API_KEY:
        print("  - No ANTHROPIC_API_KEY found in environment")

Preparing LLM evaluation batch requests (including DAGs that failed parser validation)...
Preparing evaluation batches...
Preparing requests for all 412 DAGs...
Skipped 70 non-DAG generation requests
Preparing requests for all 412 DAGs...
Skipped 70 non-DAG generation requests
Preparing requests for all 412 DAGs...
Skipped 70 non-DAG generation requests
✓ Prepared 1026 evaluation requests (342 ground + 342 baseline + 342 fine-tuned)


In [ ]:
# Execute Batch Processing and Parse Results
print("🚀 Submitting batch request to Claude API...")
print(f"   This will evaluate {len(all_batch_requests)} DAGs using Claude Sonnet 4")
print(f"   Estimated cost: ~${len(all_batch_requests) * 0.015:.2f} (at $15/1M input tokens)")
print()

# Submit batch
batch_id = processor.submit_batch(all_batch_requests)
print(f"✓ Batch submitted: {batch_id}")
print()

2025-12-30 16:44:41,775 - INFO - 🚀 Submitting batch with 824 requests...


🚀 Submitting batch request to Claude API...
   This will evaluate 824 DAGs using Claude Sonnet 4
   Estimated cost: ~$12.36 (at $15/1M input tokens)



2025-12-30 16:44:44,569 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages/batches?beta=true "HTTP/1.1 200 OK"
2025-12-30 16:44:44,573 - INFO - ✅ Batch submitted: msgbatch_01GuxFdkP6XrMjqjzxNwjCGb
2025-12-30 16:44:44,574 - INFO - ⏳ Waiting for batch msgbatch_01GuxFdkP6XrMjqjzxNwjCGb...
2025-12-30 16:44:44,753 - INFO - HTTP Request: GET https://api.anthropic.com/v1/messages/batches/msgbatch_01GuxFdkP6XrMjqjzxNwjCGb?beta=true "HTTP/1.1 200 OK"
2025-12-30 16:44:44,756 - INFO - 📊 Status: in_progress (elapsed: 0.2s)
2025-12-30 16:44:44,757 - INFO -    Progress: 0/824 (0.0%)


✓ Batch submitted: msgbatch_01GuxFdkP6XrMjqjzxNwjCGb

⏳ Waiting for batch to complete (this may take 10-30 minutes)...


2025-12-30 16:45:15,001 - INFO - HTTP Request: GET https://api.anthropic.com/v1/messages/batches/msgbatch_01GuxFdkP6XrMjqjzxNwjCGb?beta=true "HTTP/1.1 200 OK"
2025-12-30 16:45:15,005 - INFO - 📊 Status: in_progress (elapsed: 30.4s)
2025-12-30 16:45:15,006 - INFO -    Progress: 0/824 (0.0%)
2025-12-30 16:45:45,259 - INFO - HTTP Request: GET https://api.anthropic.com/v1/messages/batches/msgbatch_01GuxFdkP6XrMjqjzxNwjCGb?beta=true "HTTP/1.1 200 OK"
2025-12-30 16:45:45,263 - INFO - 📊 Status: in_progress (elapsed: 60.7s)
2025-12-30 16:45:45,263 - INFO -    Progress: 0/824 (0.0%)
2025-12-30 16:46:15,515 - INFO - HTTP Request: GET https://api.anthropic.com/v1/messages/batches/msgbatch_01GuxFdkP6XrMjqjzxNwjCGb?beta=true "HTTP/1.1 200 OK"
2025-12-30 16:46:15,518 - INFO - 📊 Status: in_progress (elapsed: 90.9s)
2025-12-30 16:46:15,519 - INFO -    Progress: 0/824 (0.0%)
2025-12-30 16:46:45,750 - INFO - HTTP Request: GET https://api.anthropic.com/v1/messages/batches/msgbatch_01GuxFdkP6XrMjqjzxNwjCGb


📥 Downloading results...


2025-12-30 16:47:46,823 - INFO - HTTP Request: GET https://api.anthropic.com/v1/messages/batches/msgbatch_01GuxFdkP6XrMjqjzxNwjCGb/results "HTTP/1.1 200 OK"
2025-12-30 16:47:47,475 - INFO - 📥 Downloaded 824 items


✓ Downloaded 824 results

📊 Parsing evaluation scores...
✓ Parsed 489 evaluation scores
⚠️ 335 results had errors or couldn't be parsed

LLM Evaluation Summary (1-5 scale):


,correctness,completeness,best_practices
model,,,
Baseline,1.98,1.78,1.82
Fine-tuned,1.78,1.57,1.76


In [ ]:
 # Wait for completion (can take 10-30 minutes for large batches)
print("⏳ Waiting for batch to complete (this may take 10-30 minutes)...")
batch = processor.wait_for_batch_completion(batch_id)
print()

# Download results
print("📥 Downloading results...")
results = processor.download_batch_results(batch_id)
print(f"✓ Downloaded {len(results)} results")
print()

# Parse results into dataframe
print("📊 Parsing evaluation scores...")
llm_eval_results = []
parse_errors = 0

for result in results:
    if result['result']['type'] == 'succeeded':
        # Parse model name and dag_id from custom_id
        # Handle format: "baseline_123" or "finetuned_123"
        custom_id = result['custom_id']
        parts = custom_id.split('_')
        model_name = parts[0]  # 'ground', 'baseline' or 'finetuned'
        dag_id = int(parts[1])
        text = result['result']['message']['content'][0]['text']
        try:
            json_match = re.search(r'\{.*\}', text, re.DOTALL)
            if json_match:
                json_str = json_match.group(0)
                eval_data = json.loads(json_str)
            else:
                raise ValueError("No JSON object found in response")
            llm_eval_results.append({
                'model': model_name,
                'dag_id': dag_id,
                'idiomatic_airflow_score': eval_data['idiomatic_airflow']['score'],
                'idiomatic_airflow_reasoning': eval_data['idiomatic_airflow']['reasoning'],
                'no_hallucination_score': eval_data['no_hallucination']['score'],
                'no_hallucination_reasoning': eval_data['no_hallucination']['reasoning'],
                'instruction_adherence_score': eval_data['instruction_adherence']['score'],
                'instruction_adherence_reasoning': eval_data['instruction_adherence']['reasoning'],
            })
        except json.JSONDecodeError:
            parse_errors += 1
    elif result['result']['type'] == 'errored':
        parse_errors += 1


df_llm = pd.DataFrame(llm_eval_results)
print(f"✓ Parsed {len(llm_eval_results)} evaluation scores")
if parse_errors > 0:
    print(f"⚠️ {parse_errors} results had errors or couldn't be parsed")

# Display summary statistics
summary = df_llm.groupby('model').agg({
    'idiomatic_airflow_score': 'mean',
    'no_hallucination_score': 'mean',
    'instruction_adherence_score': 'mean'
}).round(2)

print("\nLLM Evaluation Summary (1-5 scale):")
display(summary)

2025-12-31 11:08:27,422 - INFO - ⏳ Waiting for batch msgbatch_01Mue6HSzr5mR9g3XR1aEST9...


⏳ Waiting for batch to complete (this may take 10-30 minutes)...


2025-12-31 11:08:27,676 - INFO - HTTP Request: GET https://api.anthropic.com/v1/messages/batches/msgbatch_01Mue6HSzr5mR9g3XR1aEST9?beta=true "HTTP/1.1 200 OK"
2025-12-31 11:08:27,679 - INFO - 📊 Status: ended (elapsed: 0.3s)
2025-12-31 11:08:27,679 - INFO -    Progress: 1026/1026 (100.0%)
2025-12-31 11:08:27,679 - INFO - ✅ Batch completed with status: ended
2025-12-31 11:08:27,680 - INFO - ⬇️ Downloading results...
2025-12-31 11:08:27,876 - INFO - HTTP Request: GET https://api.anthropic.com/v1/messages/batches/msgbatch_01Mue6HSzr5mR9g3XR1aEST9?beta=true "HTTP/1.1 200 OK"



📥 Downloading results...


2025-12-31 11:08:28,160 - INFO - HTTP Request: GET https://api.anthropic.com/v1/messages/batches/msgbatch_01Mue6HSzr5mR9g3XR1aEST9/results "HTTP/1.1 200 OK"
2025-12-31 11:08:28,794 - INFO - 📥 Downloaded 1026 items


✓ Downloaded 1026 results

📊 Parsing evaluation scores...
✓ Parsed 1025 evaluation scores
⚠️ 1 results had errors or couldn't be parsed

LLM Evaluation Summary (1-5 scale):


,idiomatic_airflow_score,no_hallucination_score,instruction_adherence_score
model,,,
baseline,0.11,0.24,0.15
finetuned,0.43,0.06,0.08
ground,0.91,0.17,0.72
